Generates tables involving Shannon ENtropy and Mutual Information.

In [1]:
import matplotlib.pyplot as plt
import math
import re
import os

# mount drive -- only applies if the dataset is in the drive. otherwise, point to relevant dataset
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# drive.mount('/content/drive/')
#dir = '/content/drive/My Drive/CAS-Assignment1/GraphGeneration/simcov_data/default'
dir = '/content/drive/My Drive/CAS-Assignment1/GraphGeneration/simcov_data/fig3'




In [4]:
filename = 'simcov_3.stats'

time = []
incb = []
expr = []
apop = []
dead = []
tvas = []
ttis = []
chem = []
virs = []
chempts = []
infct = []

def makeSimpleTimeSeries(x_init, r, numSteps):
  x_old = x_init
  xs = [x_init]

  for i in range(0, numSteps):
    x_new = r * x_old * (1 - x_old)
    x_old = x_new
    xs.append(x_new)

  return xs

def getShannonEntropy(relativePops, basePop, startInd=None, endInd=None):
  popToOccurences = {} # dict from population size to number of occurences

  if startInd is None:
    startInd = 0

  if endInd is None:
    endInd = len(relativePops) - 1

  numPops = endInd - startInd + 1

  for i in range(startInd, endInd, 1):
    pop = math.floor(relativePops[i] * basePop)

    if pop in popToOccurences:
      popToOccurences[pop] = popToOccurences[pop] + 1
    else:
      popToOccurences[pop] = 1

  sum = 0

  for pop, occurenceCount in popToOccurences.items():
    probability = occurenceCount / numPops

    sum = sum + probability * math.log2(probability)

  return -sum


def getMutualInformation(relativePops1, relativePops2, basePop, startInd=None, endInd=None):
  popToOccurences = {} # dict from population size to { [1] = occurences1, [2] = occurences2 }

  if startInd is None:
    startInd = 0

  if endInd is None:
    endInd = len(relativePops1) - 1

  numPops = endInd - startInd + 1

  for i in range(startInd, endInd, 1):
    pop1 = math.floor(relativePops1[i] * basePop)
    pop2 = math.floor(relativePops2[i] * basePop)

    if pop1 in popToOccurences:
      popToOccurences[pop1][1] = popToOccurences[pop1][1] + 1
    else:
      popToOccurences[pop1] = { 1:1, 2:0 }

    if pop2 in popToOccurences:
      popToOccurences[pop2][2] = popToOccurences[pop2][2] + 1
    else:
      popToOccurences[pop2] = { 1:0, 2:1 }

  sum = 0

  for pop, occurences in popToOccurences.items():
    prob1 = occurences[1] / numPops
    prob2 = occurences[2] / numPops

    # The two runs are independent, so P(A or B) = P(A) + P(B) - P(A and B) = P(A) + P(B) - P(A) * P(B)
    probJoint = prob1 + prob2 - prob1 * prob2

    # I(X;Y) = H(X) + H(Y) - H(X or Y)
    sum = sum +\
      prob1 * math.log2( prob1 if prob1 > 0 else 1 ) +\
      prob2 * math.log2( prob2 if prob2 > 0 else 1 ) -\
      probJoint * math.log2( probJoint )

  return -sum


# map incoming data to lists for plotting
with open(os.path.join(dir, filename), "r") as f:
  for line in f:
    if line.startswith('#'):
      continue
    data = line.split('\t')
    time.append(float(data[0]) / 1440)
    incb.append(float(data[1]))
    expr.append(float(data[2]))
    apop.append(float(data[3]))
    dead.append(float(data[4]))
    tvas.append(float(data[5]))
    ttis.append(float(data[6]))
    chem.append(float(data[7]))
    virs.append(float(data[8]))
    chempts.append(float(data[9]))
    infct.append(float(data[10]))


# TODO: Display as a table image as per 2.2 specifications.

In [5]:
# 2.2: Table 1

numTimeSteps = 200
basePopulationSize = 100
divergentTimeStep = 25 # DEFINEME
x_init_a = 0.3
x_init_b = 0.300001
r_1 = 2.9 # Stable populations
r_2 = 3.5 # Periodic populations
#r_3 = 3.68887 # Chaotic populations
r_3 = 3.7 # Chaotic populations


xs_1a = makeSimpleTimeSeries(x_init_a, r_1, numTimeSteps)
xs_1b = makeSimpleTimeSeries(x_init_b, r_1, numTimeSteps)

xs_2a = makeSimpleTimeSeries(x_init_a, r_2, numTimeSteps)
xs_2b = makeSimpleTimeSeries(x_init_b, r_2, numTimeSteps)

xs_3a = makeSimpleTimeSeries(x_init_a, r_3, numTimeSteps)
xs_3b = makeSimpleTimeSeries(x_init_b, r_3, numTimeSteps)


print("n = " + str(divergentTimeStep))
print()

print("H(1..n in chaotic_a) = " + str(getShannonEntropy(xs_3a, basePopulationSize, 1, divergentTimeStep)))
print("H(n+1..N in chaotic_a) = " + str(getShannonEntropy(xs_3a, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("H(1..n in periodic_a) = " + str(getShannonEntropy(xs_2a, basePopulationSize, 1, divergentTimeStep)))
print("H(n+1..N in periodic_a) = " + str(getShannonEntropy(xs_2a, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("MI(1..n in chaotic) = " + str(getMutualInformation(xs_3a, xs_3b, basePopulationSize, 1, divergentTimeStep)))
print("MI(n+1..N in chaotic) = " + str(getMutualInformation(xs_3a, xs_3b, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()

print("MI(1..n in periodic) = " + str(getMutualInformation(xs_2a, xs_2b, basePopulationSize, 1, divergentTimeStep)))
print("MI(n+1..N in periodic) = " + str(getMutualInformation(xs_2a, xs_2b, basePopulationSize, divergentTimeStep + 1, numTimeSteps)))
print()




n = 25

H(1..n in chaotic_a) = 4.058101942183733
H(n+1..N in chaotic_a) = 5.5377237043391725

H(1..n in periodic_a) = 3.8679064420971963
H(n+1..N in periodic_a) = 1.9966970406535007

MI(1..n in chaotic) = 1.9254592981307852
MI(n+1..N in chaotic) = 1.7970570990922057

MI(1..n in periodic) = 1.9972285087343387
MI(n+1..N in periodic) = 1.904264527927898



In [ ]:
# 2.3: Table 2

basePopulationSize = 1
startInd = None
endInd = None
x_init_a = 0.3
x_init_b = 0.300001
r_1 = 2.9 # Stable populations
r_2 = 3.5 # Periodic populations
r_3 = 3.68887 # Chaotic populations

print("H(expressing cells) = " + str(getShannonEntropy(expr, basePopulationSize, startInd, endInd)))
print("H(T cells) = " + str(getShannonEntropy(ttis, basePopulationSize, startInd, endInd)))
print("MI(expressing cells, T cells) = " + str(getMutualInformation(expr, ttis, basePopulationSize, startInd, endInd)))
print()

H(expressing cells) = 14.357083847341826
H(T cells) = 0.9775030018729044
MI(expressing cells, T cells) = 0.03825978326535277



In [ ]:
# this is for fig 3
filename1 = 'simcov.stats'
#filename2 = 'simcov32.stats'
#filename3 = 'simcov33.stats'

time1 = []
expr1 = []
ttis1 = []

# map incoming data to lists for plotting
with open(os.path.join(dir, filename1), "r") as f:
  for line in f:
    if line.startswith('#'):
      continue
    data = line.split('\t')
    time1.append(float(data[0]) / 1440)
    expr1.append(float(data[2]))
    ttis1.append(float(data[6]))
time2 = []
expr2 = []
ttis2 = []
'''
with open(os.path.join(dir, filename2), "r") as f:
  for line in f:
    if line.startswith('#'):
      continue
    data = line.split('\t')
    time2.append(float(data[0]) / 1440)
    expr2.append(float(data[2]))
    ttis2.append(float(data[6]))
'''

'''
time3 = []
expr3 = []
ttis3 = []
with open(os.path.join(dir, filename3), "r") as f:
  for line in f:
    if line.startswith('#'):
      continue
    data = line.split('\t')
    time3.append(float(data[0]) / 1440)
    expr3.append(float(data[2]))
    ttis3.append(float(data[6]))
'''

'\ntime3 = []\nexpr3 = []\nttis3 = []\nwith open(os.path.join(dir, filename3), "r") as f:\n  for line in f:\n    if line.startswith(\'#\'):\n      continue\n    data = line.split(\'\t\')\n    time3.append(float(data[0]) / 1440)\n    expr3.append(float(data[2]))\n    ttis3.append(float(data[6]))\n'

In [ ]:
fig, axs = plt.subplots(3,figsize=(10,10))
#sharex=True

axs[0].plot(time1, expr1, color='blue', lw='1', label='Expressing cells')
axs[0].plot(time1, ttis1, color='red', lw='1', label='T cells')
axs[0].legend(frameon=True, fontsize=8, loc='upper left')
axs[0].set_title("Case 1:")
axs[0].set_ylabel('Cell Count')
#axs[0].set_xlabel('Time (Days)')
axs[0].set_ylim(1, 1e4)
axs[0].set_yscale('log')


axs[1].plot(time2, expr2, color='blue', lw='1', label='Expressing cells')
axs[1].plot(time2, ttis2, color='red', lw='1', label='T cells')
axs[1].legend(frameon=True, fontsize=8, loc='upper left')
axs[1].set_title("Case 2:")
axs[1].set_ylabel('Cell Count')
#axs[1].set_xlabel('Time (Days)')
axs[1].set_ylim(1, 1e5)
axs[1].set_yscale('log')

axs[2].plot(time3, expr3, color='blue', lw='1', label='Expressing cells')
axs[2].plot(time3, ttis3, color='red', lw='1', label='T cells')
axs[2].legend(frameon=True, fontsize=8, loc='upper left')
axs[2].set_title("Case 3:")
axs[2].set_ylabel('Cell Count')
axs[2].set_xlabel('Time (Days)')
axs[2].set_ylim(1, 1e6)
axs[2].set_yscale('log')

plt.show()